In [2]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

--2018-10-12 15:57:37--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 34.237.217.71, 52.55.184.84, 34.200.202.18, ...
Connecting to piazza.com (piazza.com)|34.237.217.71|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2018-10-12 15:57:37--  https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)... 143.204.98.56, 143.204.98.189, 143.204.98.61, ...
Connecting to d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)|143.204.98.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 10

In [3]:
from google.colab import files
uploaded = files.upload()


Saving Pride.txt to Pride.txt


In [21]:
import unidecode
import string
import random
import re
 
import pdb
 
all_characters = string.printable
n_characters = len(all_characters)
 
file = unidecode.unidecode(open('./text_files/alma.txt').read())
# file = unidecode.unidecode(open('Pride.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 466656


In [17]:
chunk_len = 1000
 
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]
 
print(random_chunk())

teeth. I have not passed through fire and 
death to bandy crooked words with a serving -man till the lightning falls.' 

He raised his staff. There was a roll of thunder. The sunlight was blotted 
out from the eastern windows; the whole hall became suddenly dark as night. 
The fire faded to sullen embers. Only Gandalf could be seen, standing white 
and tall before the blackened hearth. 

In the gloom they heard the hiss of Wormtongue's voice: 'Did I not 
counsel you, lord, to forbid his staff? That fool, Hbma, has betrayed us!' 
There was a flash as if lightning had cloven the roof. Then all was silent. 
Wormtongue sprawled on his face. 

'Now Thjoden son of Thengel, will you hearken to me?' said Gandalf. 'Do 
you ask for help?' He lifted his staff and pointed to a high window. There 
the darkness seemed to clear, and through the opening could be seen, high 
and far, a patch of shining sky. 'Not all is dark. Take courage, Lord of the 
Mark; for better help you will not find. No counsel

In [18]:
import torch
from torch.autograd import Variable
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)
 
print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


In [0]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [0]:
import torch
import torch.nn as nn
from torch.nn import ReLU as relu
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
# New imports
from torch.nn.parameter import Parameter
import pdb 
 
assert torch.cuda.is_available()


import os
import gzip
import tarfile
import gc
from torch.autograd import Variable
 
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
 
        # encode using embedding layer
        # set up GRU passing in number of layers parameter (nn.GRU)
        # decode output
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

 
    def forward(self, input_char, hidden):
        
        # by reviewing the documentation, construct a forward function that properly uses the output
        # of the GRU
        # return output and hidden
        
        output = self.embedding(input_char).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
 
    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [0]:
def train(inp, target):
    ## initialize hidden layers, set up gradient and loss
    hidden =  decoder.init_hidden()
    decoder_optimizer.zero_grad()
    ## 
    loss = 0
    
    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)# run the forward pass of your rnn with proper input
        loss += criterion(output, target[c].unsqueeze(0))
 
    ## calculate backwards loss and step the optimizer (globally)
    loss.backward()
    decoder_optimizer.step()
    
    ## /
 
    return loss.item() / chunk_len

In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    ## initialize hidden variable, initialize other useful variables 
    hidden = decoder.init_hidden()
    
    predicted = ""
    
    ## /
 
    prime_input = char_tensor(prime_str)
 
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
 
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden) # run your RNN/decoder forward on the input
 
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
 
        ## get character from your list of all characters, add it to your output str sequence, set input
        ## for the next pass through the model
        predicted += all_characters[top_i]
        inp = top_i
        ## /
 
    return predicted

In [0]:
class GRU(nn.Module):
    
    def __init__(input_size, hidden_size, bias=True, nonlinearity="tanh"):
        super(GRE, self).__init__()
        pass
    
    def forward(input_size, hidden_size):
        pass
    
    

In [0]:
import time
n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

In [23]:
for epoch in range(1, n_epochs + 1):
    loss_ = train(*random_training_set())       
    loss_avg += loss_
 
    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
        print(evaluate('Wh', 100), '\n')
 
    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

[101.85524725914001 (100 5%) 2.1647]
e tha wou tho inttem, wor the, beparond thest of the wat of the thesul onthept and the ch, san tou p 

[205.32700729370117 (200 10%) 1.8265]
ich unith, unto the Laint brith urto sail word thow untined thorme his eft the, the dith be whis beh 

[309.2889332771301 (300 15%) 1.9229]
at Be lamanetion onitech theher; be of the preat saintiro they the themed the Lamaited?

 Ammouce it 

[413.2393057346344 (400 20%) 1.7247]
em and Amane my in the Lall word and the peould, and there unou kned of the Lord agan to this agates 

[517.3857996463776 (500 25%) 1.7117]
eards.

 Yeard in them nory the shous they were peretion ins; ye in to said had dresword, bece the a 

[621.3371608257294 (600 30%) 1.6281]
e them, vory for they wilhed that in their did shas peapdonied of the welie overther past that their 

[725.1384043693542 (700 35%) 1.3046]
e and deavent again of preatherstiess that they ow thandting panl the words and thou lamantion unto  

[828.8221592903137 

### Here are 15 samples from the provided alma.txt file

In [25]:
for i in range(15):
    print("Sample {}\n".format(i+1))
    print(evaluate('Wh', 350), '\n\n')

Sample 1

en they staink they said in our hand were no part all agains the destroi-her who they armiers who were suffer so decerning the prorth of God, came to pass that they were whill we know and thus against the king, buth ye do be refen the people in the prisonies of Moroni be learted in the brhen all the hands even prosi;

 Therefore the sec, and they a 


Sample 2

en they was his did a by the worsed on no by the Lord him; yea, to whost on their mefled you, for they will great of then serpood to before God, in unto the words of the Lame that ye soot what prepary ast in the crised his be word of Zours of the rightom a coned they life, to pass and people, that ye ghold we they mentence from the pause the faders 


Sample 3

eding the excead cound well be, for you not bor the trupi it were did east said that they said unto them all should knowneran anto his also mald destronourst shall cast hearce which Teancum, according, I they come unto me persents that they should nertion blies 

### Here are 15 samples from the model trained with Pride and Prejudice

In [15]:
for i in range(15):
    print("Sample {}\n".format(i+1))
    print(evaluate('Wh', 200), '\n\n')

Sample 1

ing care, whend heemed their from
is neicuation, he was the before very with a could of displimentions, and I much
must such propose."

"You was of not been propition unows open."

"Yes, there intant  


Sample 2

er there wife much be rest of compance, that not have been in the chapplied Elizabeth be my my up fain have will be her my his never then me observed other see!" cruess. She has been she must be ad in 


Sample 3

ing my suble. The could been of it, a had restirued, will thecrauthing you aratulaint, was he realsured wold, we
was not be
ablight, would not up my complys as
nothins; and sempe in panther to takes d 


Sample 4

ore
in the of that you would accumpals that hear to might acquiminest incer waysure
that I do not much, the strained
the not unleded."

"You reasons, othing indered her theming of listentity; and I wa 


Sample 5

ell her, my unerned his attentitly the honet to a solly her obserfertance so pade this tway, a come well and her ask, I could naid mak